In [1]:
# Install the required libraries
!pip install langchain langchain-google-genai langgraph tavily-python pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requ

In [1]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import os
import operator
import yaml
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END

In [3]:
with open("api_keys (2) (1) (2).yml", "r") as f:
    config = yaml.safe_load(f)
config.keys()

dict_keys(['GEMINI_API_KEY', 'NGORK_AUTH_TOKEN', 'COHERE_API_KEY', 'OPENAI_KEY', 'LLAMA_KEY', 'TAVILY_API_KEY'])

In [4]:
os.environ["GOOGLE_API_KEY"] = config["GEMINI_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]

In [5]:


# --- API Key Management (using YAML) ---
# Load API keys from the YAML file.
# The file must be named 'api_keys.yml' and uploaded to the Colab root directory.


# --- Agent Core Components ---
# Initialize the Gemini model
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# Define the search tool
tavily_tool = TavilySearchResults(max_results=5)

# Wrap the tool in a LangChain tool decorator
@tool
def get_competitor_info(location: str):
    """
    Use this tool to get information about clothing store competitors in a given location,
    including their footfall and busy hours.
    """
    # Use .invoke() to run the tool and get results
    return tavily_tool.invoke(f"clothing store competitors in {location} footfall and busy hours")

tools = [get_competitor_info]

# Bind the tools to the model
model_with_tools = model.bind_tools(tools)

# --- Define the Graph State and Nodes ---
class AgentState(TypedDict):
    """Represents the state of our graph."""
    messages: Annotated[List[BaseMessage], operator.add]
    intermediate_steps: Annotated[List[Union[AgentAction, AgentFinish]], operator.add]

# Define a node for the LLM
def run_model(state):
    """Node that invokes the LLM to decide the next action."""
    messages = state['messages']
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}

# Define a node for tool execution
def execute_tools(state):
    """Node that executes the tools recommended by the LLM."""
    last_message = state['messages'][-1]

    # Extract tool calls from the last message
    tool_calls = last_message.tool_calls

    if not tool_calls:
        return {"messages": [HumanMessage(content="No tool calls detected.")]}

    tool_results = []
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_input = tool_call["args"]

        if tool_name == "get_competitor_info":
            result = get_competitor_info.invoke(tool_input)
            tool_results.append(result)
        else:
            raise ValueError(f"Unknown tool: {tool_name}")

    # Return the results as an intermediate step to be processed by the LLM
    return {"intermediate_steps": [("tool_response", str(tool_results))]}

# --- Define the Graph and Edges ---
def should_continue(state):
    """Decides whether to continue the graph or end."""
    last_message = state['messages'][-1]
    # Check if the message contains tool calls. Gemini stores them in `tool_calls`.
    if last_message.tool_calls:
        return "continue"
    else:
        # If there are no tool calls, it's a final response.
        return "end"

# Build the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("call_model", run_model)
workflow.add_node("call_tool", execute_tools)

# Set the entry point
workflow.set_entry_point("call_model")

# Add edges
workflow.add_conditional_edges(
    "call_model",
    should_continue,
    {"continue": "call_tool", "end": END}
)
workflow.add_edge("call_tool", "call_model")

# Compile the graph
app = workflow.compile()

# --- Example Usage ---
location = "Koramangala, Bangalore"
user_query = f"Generate a detailed report on clothing store competitors in {location}, including footfall trends and peak hours."

# Run the graph and print the final state
print("\n--- Running the LangGraph Agent ---")
for step in app.stream({"messages": [HumanMessage(content=user_query)]}, stream_mode="values"):
    print(step)

/tmp/ipython-input-728609002.py:11: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)



--- Running the LangGraph Agent ---
{'messages': [HumanMessage(content='Generate a detailed report on clothing store competitors in Koramangala, Bangalore, including footfall trends and peak hours.', additional_kwargs={}, response_metadata={})], 'intermediate_steps': []}
{'messages': [HumanMessage(content='Generate a detailed report on clothing store competitors in Koramangala, Bangalore, including footfall trends and peak hours.', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_competitor_info', 'arguments': '{"location": "Koramangala, Bangalore"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--98371371-b07d-447a-8238-d5786dae627e-0', tool_calls=[{'name': 'get_competitor_info', 'args': {'location': 'Koramangala, Bangalore'}, 'id': '4f4e3673-218b-489a-8ece-396ffbbc2d01', 'type': 'tool_call'}

In [8]:
print("\n--- Running the LangGraph Agent ---")

printed_human = False  # track if we already printed the human query

for step in app.stream({"messages": [HumanMessage(content=user_query)]}, stream_mode="values"):
    messages = step.get("messages", [])
    for msg in messages:
        role = msg.__class__.__name__.replace("Message", "")  # Human / AI
        content = msg.content.strip()

        if not content:
            continue  # skip empty

        # print human only once
        if role == "Human":
            if not printed_human:
                print(f"\n[{role}] {content}")
                printed_human = True
        else:
            print(f"\n[{role}] {content}")



--- Running the LangGraph Agent ---

[Human] Generate a detailed report on clothing store competitors in Koramangala, Bangalore, including footfall trends and peak hours.

[AI] Here's a detailed report on clothing store competitors in Koramangala, Bangalore, including footfall trends and peak hours:

**Competitor Analysis: Koramangala, Bangalore**

**Overview:**
Koramangala, a vibrant and upscale locality in Bangalore, is a hub for fashion and retail. The area boasts a diverse range of clothing stores, from international brands to local boutiques, catering to a wide demographic. Competition is intense, with stores constantly vying for customer attention through unique offerings, promotions, and customer service.

**Key Competitors:**

*   **H&M:** A popular international fast-fashion retailer, H&M in Koramangala attracts a significant young demographic.
    *   **Footfall Trends:** Consistently high, especially on weekends and evenings. Weekday afternoons see moderate footfall.
    * 